# 프로젝트 : Movielens 영화 추천 실습

추천시스템의 MNIST라고 부를만한 [Movielens](https://grouplens.org/datasets/movielens/1m/) 데이터셋과
<br> [implicit](https://github.com/benfred/implicit) 패키지의 als(AlternatingLeastSquares) 모델을 활용하여
<br>영화 추천 시스템을 만들어보자.

#### ✔︎ 모듈 불러오기

In [1]:
import pandas as pd
import os
import numpy as np
from scipy.sparse import csr_matrix
from implicit.als import AlternatingLeastSquares

---

## 1. 데이터 준비와 전처리

### 1) Ratings 데이터

Movielens 데이터는 ratings.dat 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있다.

In [2]:
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외한다.

In [3]:
# 3점 이상만 남긴다.
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


별점을 시청횟수로 해석해서 생각해줄 예정이기 때문에 컬럼 이름을 바꿔준다.

In [4]:
# ratings 컬럼의 이름을 counts로 바꿈
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [5]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

### 2) Movies 데이터

In [6]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옴
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


### 3) Ratings와 Movies 데이터 결합한 Movie_ratings 데이터 만들기

In [7]:
# Ratings + Movies 데이터프레임 결합 (movies 데이터의 movie_id를 인덱스로 만들어준 후 병합)
movie_ratings = ratings.join(movies.set_index('movie_id'), on='movie_id')
movie_ratings.head()

,user_id,movie_id,counts,timestamp,title,genre
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance
3,1,3408,4,978300275,Erin Brockovich (2000),Drama
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy


In [8]:
# timestamp 컬럼 제거
movie_ratings = movie_ratings.drop(columns=['timestamp'])
movie_ratings.head()

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,661,3,James and the Giant Peach (1996),Animation|Children's|Musical
2,1,914,3,My Fair Lady (1964),Musical|Romance
3,1,3408,4,Erin Brockovich (2000),Drama
4,1,2355,5,"Bug's Life, A (1998)",Animation|Children's|Comedy


---
## 2. 데이터 분석

### 1) 데이터 전체적으로 살펴보기

In [9]:
# movie_ratings에 있는 유니크한 영화 개수
movie_ratings['movie_id'].nunique()

3628

In [10]:
# movie_ratings에 있는 유니크한 사용자 수
movie_ratings['user_id'].nunique()

6039

In [11]:
# 가장 인기 있는 영화 30개 (인기순)
movie_count = movie_ratings.groupby('title')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [12]:
# 유저별 몇 편의 영화를 봤는지에 대한 통계
user_count = movie_ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

평균적으로 한 명의 유저 당 138편의 영화를 보고 평점을 매겼고, 최소 1편, 최대 1968편을 본 유저가 있다는 것을 알 수 있다.

### 2) 장르별로 데이터 살펴보기

우선 어떤 식으로 장르를 분류해놨는지 살펴봐야겠다.

In [13]:
movie_ratings['genre'].unique()

array(['Drama', "Animation|Children's|Musical", 'Musical|Romance',
       "Animation|Children's|Comedy", 'Action|Adventure|Comedy|Romance',
       'Action|Adventure|Drama', 'Comedy|Drama',
       "Adventure|Children's|Drama|Musical", 'Musical', 'Comedy',
       "Animation|Children's", 'Comedy|Fantasy', 'Animation',
       'Comedy|Sci-Fi', 'Drama|War', 'Romance',
       "Animation|Children's|Musical|Romance",
       "Children's|Drama|Fantasy|Sci-Fi", 'Drama|Romance',
       'Animation|Comedy|Thriller',
       "Adventure|Animation|Children's|Comedy|Musical",
       "Animation|Children's|Comedy|Musical", 'Thriller',
       'Action|Crime|Romance', 'Action|Adventure|Fantasy|Sci-Fi',
       "Children's|Comedy|Musical", 'Action|Drama|War',
       "Children's|Drama", 'Crime|Drama|Thriller', 'Action|Crime|Drama',
       'Action|Adventure|Mystery', 'Crime|Drama',
       'Action|Adventure|Sci-Fi|Thriller',
       'Action|Adventure|Romance|Sci-Fi|War', 'Action|Thriller',
       'Action|Drama', 'Co

한 영화에 여러 장르가 섞여있으니 어떤 장르의 종류가 있는지 파악하기 어렵다. 장르의 종류만 뽑아내보자.

In [14]:
genre_list = []
for genre_each_movie in movie_ratings['genre']:
    genre_each_movie = genre_each_movie.split('|')
    for genre_uniq in genre_each_movie:
        genre_list.append(genre_uniq)
genre_list = set(genre_list)
genre_list

{'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western'}

총 18개의 장르가 존재한다. 내가 관심있는 장르별로 인기 있는 영화들을 살펴봐야겠다.

**Romance**

In [15]:
movie_ratings[movie_ratings['genre'].str.contains('romance', case=False)]

,user_id,movie_id,counts,title,genre
2,1,914,3,My Fair Lady (1964),Musical|Romance
5,1,1197,3,"Princess Bride, The (1987)",Action|Adventure|Comedy|Romance
24,1,2340,3,Meet Joe Black (1998),Romance
25,1,48,5,Pocahontas (1995),Animation|Children's|Musical|Romance
27,1,1721,4,Titanic (1997),Drama|Romance
...,...,...,...,...,...
1000147,6040,2369,3,Desperately Seeking Susan (1985),Comedy|Romance
1000148,6040,899,4,Singin' in the Rain (1952),Musical|Romance
1000158,6040,2396,3,Shakespeare in Love (1998),Comedy|Romance
1000197,6040,2020,3,Dangerous Liaisons (1988),Drama|Romance


In [16]:
romance_movies = movie_ratings[movie_ratings['genre'].str.contains('romance', case=False)]

In [17]:
romance_movies = romance_movies.groupby('title')['user_id'].count()
romance_movies.sort_values(ascending=False).head(30)

title
Star Wars: Episode VI - Return of the Jedi (1983)    2716
Princess Bride, The (1987)                           2252
Shakespeare in Love (1998)                           2213
Groundhog Day (1993)                                 2121
Forrest Gump (1994)                                  2022
Casablanca (1942)                                    1623
When Harry Met Sally... (1989)                       1487
Speed (1994)                                         1442
Edward Scissorhands (1990)                           1285
Titanic (1997)                                       1270
True Lies (1994)                                     1254
Annie Hall (1977)                                    1248
Romancing the Stone (1984)                           1239
Jerry Maguire (1996)                                 1239
Graduate, The (1967)                                 1223
Clueless (1995)                                      1207
Four Weddings and a Funeral (1994)                   1126
Crying G

**Thriller**

In [18]:
thriller_movies = movie_ratings[movie_ratings['genre'].str.contains('thriller', case=False)]
thriller_movies = thriller_movies.groupby('title')['user_id'].count()
thriller_movies.sort_values(ascending=False).head(30)

title
Terminator 2: Judgment Day (1991)    2509
Silence of the Lambs, The (1991)     2498
Matrix, The (1999)                   2434
Sixth Sense, The (1999)              2385
Fargo (1996)                         2371
L.A. Confidential (1997)             2210
Terminator, The (1984)               2019
Fugitive, The (1993)                 1941
Alien (1979)                         1920
Total Recall (1990)                  1786
Usual Suspects, The (1995)           1744
Aliens (1986)                        1720
Die Hard (1988)                      1593
Hunt for Red October, The (1990)     1587
2001: A Space Odyssey (1968)         1568
Abyss, The (1989)                    1555
Speed (1994)                         1442
North by Northwest (1959)            1298
Rock, The (1996)                     1213
Psycho (1960)                        1210
Taxi Driver (1976)                   1184
Reservoir Dogs (1992)                1176
Predator (1987)                      1151
Chinatown (1974)            

**Drama**

In [19]:
drama_movies = movie_ratings[movie_ratings['genre'].str.contains('drama', case=False)]
drama_movies = drama_movies.groupby('title')['user_id'].count()
drama_movies.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Saving Private Ryan (1998)                               2561
Silence of the Lambs, The (1991)                         2498
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Schindler's List (1993)                                  2257
Shawshank Redemption, The (1994)                         2194
Godfather, The (1972)                                    2167
E.T. the Extra-Terrestrial (1982)                        2102
Pulp Fiction (1994)                                      2030
Gladiator (2000)                                         1798
Stand by Me (1986)                                       1728
One Flew Over the Cuckoo's Nest (1975)                   1680
Wizard of Oz, The (1939)                                 1650
Godfather: Part II, The (1974)                           1624
Ca

**Musical**

In [20]:
musical_movies = movie_ratings[movie_ratings['genre'].str.contains('musical', case=False)]
musical_movies = musical_movies.groupby('title')['user_id'].count()
musical_movies.sort_values(ascending=False).head(30)

title
Wizard of Oz, The (1939)                  1650
Blues Brothers, The (1980)                1244
Aladdin (1992)                            1228
This Is Spinal Tap (1984)                 1042
Lion King, The (1994)                     1029
Beauty and the Beast (1991)                960
Little Mermaid, The (1989)                 926
Mary Poppins (1964)                        917
Rocky Horror Picture Show, The (1975)      911
Nightmare Before Christmas, The (1993)     869
Fantasia (1940)                            843
Lady and the Tramp (1955)                  804
Sound of Music, The (1965)                 776
Little Shop of Horrors (1986)              746
Singin' in the Rain (1952)                 721
Snow White and the Seven Dwarfs (1937)     707
West Side Story (1961)                     702
Grease (1978)                              673
Producers, The (1968)                      629
Jungle Book, The (1967)                    618
My Fair Lady (1964)                        611
Doors, 

---

## 3. 내가 선호하는 영화 추가

선호하는 영화들의 데이터를 살펴보자.

In [21]:
movie_ratings[movie_ratings['title'].str.contains(
    'When Harry Met Sally|Titanic|Godfather|Before|Shawshank', case=False)]

,user_id,movie_id,counts,title,genre
27,1,1721,4,Titanic (1997),Drama|Romance
172,2,318,5,"Shawshank Redemption, The (1994)",Drama
270,5,215,3,Before Sunrise (1995),Drama|Romance
350,5,318,3,"Shawshank Redemption, The (1994)",Drama
391,5,551,4,"Nightmare Before Christmas, The (1993)",Children's|Comedy|Musical
...,...,...,...,...,...
999847,6039,858,4,"Godfather, The (1972)",Action|Crime|Drama
999991,6040,318,4,"Shawshank Redemption, The (1994)",Drama
1000042,6040,1221,4,"Godfather: Part II, The (1974)",Action|Crime|Drama
1000123,6040,1307,5,When Harry Met Sally... (1989),Comedy|Romance


좀더 깔끔하게 찾을 수 있는 방법이 있을텐데, 일단은 이렇게 해서 내가 추가하고 싶은 영화들의 movie_id는 대충 알 수 있었다.
<br> **When Harry Met Sally... (1989)** : 1307
<br> **Titanic (1997)** : 1721
<br> **Godfather, The (1972)** : 858
<br> **Before Sunrise (1995)** : 215
<br> **Shawshank Redemption, The (1994)** : 318

선호하는 영화 5개를 원본 데이터프레임에 추가해보자.

In [22]:
my_movies = ['When Harry Met Sally... (1989)',
               'Titanic (1997)',
               'Godfather, The (1972)', 
               'Before Sunrise (1995)', 
               'Shawshank Redemption, The (1994)']
my_movies_id = [1307, 1721, 858, 215, 318]
my_movies_genre = ['Comedy|Romance', 
                   'Drama|Romance', 
                   'Action|Crime|Drama',
                   'Drama|Romance',
                   'Drama']

In [23]:
my_movielist = pd.DataFrame({'user_id': ['ksh']*5,
                             'movie_id': my_movies_id,
                             'counts': [5]*5, 
                             'title': my_movies,
                             'genre': my_movies_genre})

if not movie_ratings.isin({'user_id':['ksh']})['user_id'].any():
    movie_ratings = movie_ratings.append(my_movielist)
    
movie_ratings.tail(10)

,user_id,movie_id,counts,title,genre
1000203,6040,1090,3,Platoon (1986),Drama|War
1000205,6040,1094,5,"Crying Game, The (1992)",Drama|Romance|War
1000206,6040,562,5,Welcome to the Dollhouse (1995),Comedy|Drama
1000207,6040,1096,4,Sophie's Choice (1982),Drama
1000208,6040,1097,4,E.T. the Extra-Terrestrial (1982),Children's|Drama|Fantasy|Sci-Fi
0,ksh,1307,5,When Harry Met Sally... (1989),Comedy|Romance
1,ksh,1721,5,Titanic (1997),Drama|Romance
2,ksh,858,5,"Godfather, The (1972)",Action|Crime|Drama
3,ksh,215,5,Before Sunrise (1995),Drama|Romance
4,ksh,318,5,"Shawshank Redemption, The (1994)",Drama


---
## 4. CSR matrix 만들기

### 1) Indexing

CSR matrix를 만들어주기 전에 indexing을 해줄 필요가 있다.

In [24]:
# 고유한 유저, 영화, 장르를 찾아내는 코드
user_unique = movie_ratings['user_id'].unique()
movie_unique = movie_ratings['title'].unique()
genre_unique = movie_ratings['genre'].unique()

# 유저, 영화, 장르를 고유한 값의 개수만큼 indexing
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}
genre_to_idx = {v:k for k,v in enumerate(genre_unique)}

In [25]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['ksh'])
print(movie_to_idx['Before Sunrise (1995)'])
print(genre_to_idx['Drama'])
print(genre_to_idx['Drama|Romance'])

6039
213
0
18


In [26]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해본다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거한다. 
temp_user_data = movie_ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(movie_ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    movie_ratings['user_id'] = temp_user_data   # movie_ratings['user_id']을 인덱싱된 Series로 교체 
else:
    print('user_id column indexing Fail!!')

# movie_to_idx을 통해 title 컬럼도 동일한 방식으로 인덱싱 
temp_movie_data = movie_ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(movie_ratings):
    print('title column indexing OK!!')
    movie_ratings['title'] = temp_movie_data
else:
    print('title column indexing Fail!!')

# genre_to_idx을 통해 genre 컬럼도 동일한 방식으로 인덱싱 
temp_genre_data = movie_ratings['genre'].map(genre_to_idx.get).dropna()
if len(temp_genre_data) == len(movie_ratings):
    print('genre column indexing OK!!')
    movie_ratings['genre'] = temp_genre_data
else:
    print('genre column indexing Fail!!')
    
movie_ratings

user_id column indexing OK!!
title column indexing OK!!
genre column indexing OK!!


,user_id,movie_id,counts,title,genre
0,0,1193,5,0,0
1,0,661,3,1,1
2,0,914,3,2,2
3,0,3408,4,3,0
4,0,2355,5,4,3
...,...,...,...,...,...
0,6039,1307,5,488,45
1,6039,1721,5,27,18
2,6039,858,5,607,29
3,6039,215,5,213,18


사실 여기까지만 해도 장르 컬럼이 모델 훈련에 도움이 될것이란 생각에 추가적으로 계속 시간을 써서 인덱싱까지 했지만, CSR matrix를 만들 때 어차피 두 개의 특성(feature)만 사용된다는 것을 깨닫고 장르가 쓸모없음을 깨달았다. 그러나 나중에 무언가 다시 시도해볼 때에 도움이 되었으면 좋겠다는 생각에서 남겨둔다.

### 2) CSR matrix 생성

In [27]:
from scipy.sparse import csr_matrix

num_user = movie_ratings['user_id'].nunique()
num_movie = movie_ratings['title'].nunique()

csr_data = csr_matrix((movie_ratings['counts'], (movie_ratings.user_id, movie_ratings.title)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

---

## 5. ALS model 구성하여 훈련시키기

AlternatingLeastSquares 모델을 직접 구성하여 훈련시켜보자.

In [28]:
from implicit.als import AlternatingLeastSquares

# implicit 라이브러리에서 권장하고 있는 부분이므로 학습 내용과는 무관
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [29]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

- factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지
- regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지
- use_gpu : GPU를 사용할 것인지
- iterations : epochs와 같은 의미. 데이터를 몇 번 반복해서 학습할 것인지

In [30]:
# als 모델은 input으로 item X user 꼴의 matrix를 받기 때문에 Transpose해준다.
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [31]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

---

## 6. 훈련된 모델이 예측한 나의 선호도 파악

In [32]:
# 훈련된 모델이 만든 나의 벡터와 영화의 벡터 구하기
ksh, harrysally = user_to_idx['ksh'], movie_to_idx['When Harry Met Sally... (1989)']
ksh_vector, harrysally_vector = als_model.user_factors[ksh], als_model.item_factors[harrysally]

In [33]:
ksh_vector

array([-0.2802196 , -0.26783526,  0.23932311, -0.17340185, -0.10914013,
        0.32860073, -0.02470351,  0.19879526,  0.3148389 ,  0.76237386,
        0.2931195 , -0.6623077 , -0.05814015,  0.5059682 , -0.51987123,
       -0.3922641 ,  0.17650613,  0.44144577, -0.14209267,  0.01543857,
        0.13208072, -0.27454528,  0.12350249,  0.36932278, -0.03987431,
       -0.23522574, -0.11177039, -0.06428006,  0.1941185 , -0.09709091,
       -0.40497762,  0.50500244,  0.20722719, -0.08869831, -0.1671783 ,
        0.24191411,  0.63392675,  0.19372137, -0.13068   , -0.5609242 ,
        0.51847905,  0.24869823,  0.17038104, -0.55257934,  0.16541915,
       -0.14402887,  0.11609458, -0.32998183,  0.20204453,  0.945887  ,
        0.6765706 ,  0.5439616 , -0.00120888,  0.2555577 ,  0.6199387 ,
        0.5634515 ,  0.23374334, -0.9863349 ,  0.27871728,  0.53751904,
       -0.3455263 ,  0.16930625,  0.08203509, -0.92262053, -0.08210427,
       -0.46048766,  0.8764522 ,  0.07850658, -0.09670198, -0.58

In [34]:
harrysally_vector

array([-1.12167727e-02,  1.21935466e-02,  1.02083646e-02, -2.49026925e-03,
        1.00168278e-02,  8.86143185e-04,  1.25396615e-02, -8.45939619e-04,
        3.78033699e-04,  9.23457649e-03,  1.74427684e-02, -1.51049644e-02,
        7.41876941e-03,  1.46128116e-02, -4.25600307e-03, -2.20979806e-02,
        5.99082187e-03,  7.58827245e-03,  2.09751562e-03,  1.22073963e-02,
        2.66980119e-02, -3.69610754e-03,  8.01541284e-03, -1.82683654e-02,
        2.90084593e-02,  2.13786867e-02, -5.19963032e-05,  2.52362527e-02,
        1.95672512e-02, -8.32179654e-03, -1.89809594e-02,  3.58355679e-02,
        3.67465876e-02, -3.85902799e-03,  1.97851863e-02,  2.42417362e-02,
       -1.05647184e-03,  1.52649470e-02, -1.49557162e-02,  1.14693362e-02,
       -7.20409676e-03,  7.93151651e-03,  3.78678404e-02, -1.62320845e-02,
       -2.59033288e-03,  4.10149479e-03,  1.08587211e-02,  5.46410365e-06,
       -2.55750921e-02,  3.60568352e-02,  7.87871890e-03,  8.51148553e-03,
        3.53202713e-03, -

In [35]:
# 두 벡터의 내적 구하기
np.dot(ksh_vector, harrysally_vector)

0.5992108

아마 선호 리스트에 있는 영화들이 전부 비슷한 영화가 아니라서 내적 값이 낮을 수도 있겠다는 생각을 했다.
<br> 선호 리스트에 있지만 조금 다른 성격인 '대부' 영화와의 내적 값도 구해봐야겠다.

In [36]:
godfather = movie_to_idx['Godfather, The (1972)']
godfather_vector = als_model.item_factors[godfather]
np.dot(ksh_vector, godfather_vector)

0.73907024

이 정도면 꽤 높은 것 같다.<br>
선호 리스트 내의 나머지 영화들과의 내적 값을 다 구해봐야겠다.


In [37]:
titanic = movie_to_idx['Titanic (1997)']
titanic_vector = als_model.item_factors[titanic]
np.dot(ksh_vector, titanic_vector)

0.75819427

In [38]:
beforesr = movie_to_idx['Before Sunrise (1995)']
beforesr_vector = als_model.item_factors[beforesr]
np.dot(ksh_vector, beforesr_vector)

0.238681

In [39]:
shawshank = movie_to_idx['Shawshank Redemption, The (1994)']
shawshank_vector = als_model.item_factors[shawshank]
np.dot(ksh_vector, shawshank_vector)

0.7104324

'비포 선라이즈' 빼고는 다 내적 값들이 꽤 높다.
<br>선호하지 않는 영화와의 벡터 내적도 구해서 판단해봐야겠다.

In [40]:
jumanji = movie_to_idx['Jumanji (1995)']
jumanji_vector = als_model.item_factors[jumanji]
np.dot(ksh_vector, jumanji_vector)

-0.06380094

음수가 나오는 것을 보니 그렇게 나쁘진 않은 듯 하다.

---
## 7. 내가 좋아하는 영화와 비슷한 영화 추천

In [41]:
favorite_movie = 'When Harry Met Sally... (1989)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(488, 1.0000001),
 (13, 0.46663824),
 (169, 0.41589022),
 (961, 0.38222808),
 (1144, 0.37713954),
 (369, 0.3634962),
 (19, 0.34664577),
 (3079, 0.34029678),
 (786, 0.33941895),
 (946, 0.33535925),
 (3008, 0.32969403),
 (3182, 0.32881036),
 (1155, 0.3275481),
 (3201, 0.32688415),
 (3199, 0.32588887)]

In [42]:
# movie_to_idx 를 뒤집어, index로부터 movie 이름을 얻는 dict를 생성
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['When Harry Met Sally... (1989)',
 "Ferris Bueller's Day Off (1986)",
 'Fish Called Wanda, A (1988)',
 'Say Anything... (1989)',
 'Bull Durham (1988)',
 'Witness (1985)',
 'Big (1988)',
 'Free Willy 3: The Rescue (1997)',
 'Broadcast News (1987)',
 'Parenthood (1989)',
 'House Party 3 (1994)',
 'Cabinet of Dr. Ramirez, The (1991)',
 'Risky Business (1983)',
 'Allnighter, The (1987)',
 'Garbage Pail Kids Movie, The (1987)']

In [43]:
# 비슷한 영화를 알려주는 함수
def get_similar_movie(movie_title: str):
    movie_id = movie_to_idx[movie_title]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

In [44]:
get_similar_movie('Titanic (1997)')

['Titanic (1997)',
 "You've Got Mail (1998)",
 'Real Blonde, The (1997)',
 'Snow Day (2000)',
 'Saragossa Manuscript, The (Rekopis znaleziony w Saragossie) (1965)',
 'Walking Dead, The (1995)',
 'Paris, France (1993)',
 'Held Up (2000)',
 'City of Angels (1998)',
 'Gold Diggers: The Secret of Bear Mountain (1995)']

In [45]:
get_similar_movie('Godfather, The (1972)')

['Godfather, The (1972)',
 'Godfather: Part II, The (1974)',
 'Godfather: Part III, The (1990)',
 'French Connection, The (1971)',
 'Boys, The (1997)',
 "Squanto: A Warrior's Tale (1994)",
 'Niagara, Niagara (1997)',
 'Total Eclipse (1995)',
 "Star Maker, The (Uomo delle stelle, L') (1995)",
 'Held Up (2000)']

In [46]:
get_similar_movie('Before Sunrise (1995)')

['Before Sunrise (1995)',
 'Kicking and Screaming (1995)',
 'Heavy (1995)',
 'Barcelona (1994)',
 'Amateur (1994)',
 'Pillow Book, The (1995)',
 'Guinevere (1999)',
 'Hotel de Love (1996)',
 'Eight Days a Week (1997)',
 'What Happened Was... (1994)']

역시.. 위에서 봤을 때 '비포 선라이즈'에 대해서 훈련이 잘 안되었기에 나온 영화들도 다 유사하지 않은 듯 싶다.

In [47]:
get_similar_movie('Shawshank Redemption, The (1994)')

['Shawshank Redemption, The (1994)',
 'Silence of the Lambs, The (1991)',
 'Pulp Fiction (1994)',
 'GoodFellas (1990)',
 'Good Will Hunting (1997)',
 'Fargo (1996)',
 "Schindler's List (1993)",
 'Abominable Snowman, The (1957)',
 'Brother Minister: The Assassination of Malcolm X (1994)',
 'Steel (1997)']

전체적으로 봤을 때는 아무래도 내가 선호리스트에 넣은 영화들이 전부 장르가 'Drama'를 포함하고 있다보니 넓은 의미의 Drama에 속하는 영화들이 많이 나오는 것 같다.

---
## 8. 내가 가장 좋아할 만한 영화들을 추천 받기

In [48]:
user = user_to_idx['ksh']
# recommend에서는 user*item CSR Matrix를 받음
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(380, 0.549508),
 (121, 0.32259035),
 (222, 0.26636356),
 (120, 0.26102972),
 (22, 0.24442181),
 (23, 0.24029079),
 (0, 0.23740897),
 (100, 0.21402471),
 (384, 0.20781122),
 (48, 0.19946867),
 (110, 0.19940826),
 (236, 0.18643276),
 (361, 0.18572938),
 (99, 0.18343696),
 (1509, 0.18032329),
 (415, 0.17805046),
 (44, 0.17788693),
 (143, 0.1721884),
 (269, 0.17158279),
 (360, 0.16631842)]

In [49]:
# 인덱스를 영화 제목으로 변환
[idx_to_movie[i[0]] for i in movie_recommended]

['Godfather: Part II, The (1974)',
 'Silence of the Lambs, The (1991)',
 'Pulp Fiction (1994)',
 'Raiders of the Lost Ark (1981)',
 'Back to the Future (1985)',
 "Schindler's List (1993)",
 "One Flew Over the Cuckoo's Nest (1975)",
 'Amadeus (1984)',
 'Jerry Maguire (1996)',
 'Saving Private Ryan (1998)',
 'Groundhog Day (1993)',
 'Speed (1994)',
 'Casablanca (1942)',
 'American Beauty (1999)',
 'English Patient, The (1996)',
 'Cyrano de Bergerac (1990)',
 'Star Wars: Episode IV - A New Hope (1977)',
 'Gone with the Wind (1939)',
 'GoodFellas (1990)',
 'Dirty Dancing (1987)']

In [50]:
# 사용자가 기록을 남긴 데이터가 특정 영화 추천에 기여한 정도를 확인
recommended = movie_to_idx['Jerry Maguire (1996)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Shawshank Redemption, The (1994)', 0.08939266873224008),
 ('Titanic (1997)', 0.06652516776484421),
 ('When Harry Met Sally... (1989)', 0.04468346385072301),
 ('Before Sunrise (1995)', 0.004531150360652448),
 ('Godfather, The (1972)', 0.0016739897803242916)]

어느 정도 드라마의 장르로서 다 비슷하기 때문에 기여도도 비슷한 것 같다.

In [51]:
recommended = movie_to_idx['Silence of the Lambs, The (1991)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Shawshank Redemption, The (1994)', 0.2970084431707042),
 ('When Harry Met Sally... (1989)', 0.04031888907632661),
 ('Godfather, The (1972)', -0.0011773857986774512),
 ('Titanic (1997)', -0.0013335785511744479),
 ('Before Sunrise (1995)', -0.01516501540724724)]

'양들의 침묵'은 '쇼생크 탈출' 때문에 추천된 바가 큰 것 같다.

In [52]:
recommended = movie_to_idx['Godfather: Part II, The (1974)']
explain = als_model.explain(user, csr_data, itemid=recommended)

[(idx_to_movie[i[0]], i[1]) for i in explain[1]]

[('Godfather, The (1972)', 0.602169656786381),
 ('Shawshank Redemption, The (1994)', 0.012884209040845107),
 ('Before Sunrise (1995)', -0.001741063837238645),
 ('When Harry Met Sally... (1989)', -0.020372549557015472),
 ('Titanic (1997)', -0.050491735675381316)]

역시 전작인 '대부1'의 영향이 제일 크고, 정통 로맨스 드라마인 '타이타닉'과 '해리가 샐리를 만났을 때', '비포 선라이즈'와의 유사도는 음수인 것을 봤을 때 확실한 성향을 띄어야 좀 제대로 결과가 나오는 것 같다.

---

# 회고

#### 데이터셋에 관하여
- 일단 가장 기본적인 영화 데이터셋이 Movielens 데이터셋을 사용하니 제일 별로였던 점은 최근 영화가 하나도 없다는 것이었다. 7,80년대는 물론이고 아주 초창기의 30년대 영화까지 있는데 2000년 이후의 영화들이 없어서 아쉬웠다. 선호 영화 리스트에도 엄청 유명한 명작들 위주로만 넣을 수 밖에 없었고, 그러다보니 확실하게 내가 선호하는 영화들보다는 드라마 장르 영화 위주로 넣어서 결과가 아쉽게 나올 수 밖에 없었던 것 같다. 나중에 꼭 최신 영화들까지 있는 데이터셋, 더 많은 자료가 포함된 데이터셋으로 꼭 다시 해보고 싶다.<br><br>
- ratings 데이터셋과 movies 데이터셋을 처음에는 병합하지 않고 따로 진행하다가, 중간에 너무 번거로울 일이 많이 생겼다. 나의 선호 영화 리스트를 추가해줄 때도 애매했고, 나중에 CSR matrix를 만들어줄 때도 계속 오류가 났다. 그래서 다시 처음으로 돌아가서 두 데이터셋을 병합해서 하나의 데이터셋으로 만들어주고 진행하니 훨씬 편했다. 두 데이터셋의 공통인 movie_id를 기준으로 병합해주었는데, 앞으로 이런 데이터셋 병합을 더 연습할 일이 많이 있었으면 좋겠다.

#### 모델 생성, 훈련 과정에 관하여
- 우선 이번에는 가장 기본적인 추천 시스템을 연습해보는 것이 주 목적이었기에, CSR matrix를 사용하여 ALS 모델로 학습하고 결과물을 내는 것으로 만족해야 했는데, 그렇기에 여러 아쉬움이 생긴 것 같다. 첫 번째로 데이터셋에서 정말 중요한 특성 중에 하나인 장르를 써먹지도 못하고 그냥 참고용으로만 소비되는 것이 아쉬웠다. 넷플릭스나 왓챠에서 추천해주는 방식을 보면 가장 기준이 되는 것이 장르인데, CSR matrix에서는 단순히 두 특성을 연결시켜서 내적값을 계산하기 때문에 그 두 특성은 유저와 영화 제목이 차지할 수밖에 없고, 장르가 끼어들 틈이 없던 것이다. 바보같이 힘들게 장르까지 인덱싱 다해주고 나서 깨달아서 더 아쉬움이 컸다. CSR이 아닌 다른 방식으로 추천시스템을 만들 때는 꼭 장르가 중요한 특성으로서 들어가야 할 것이고, 그런 방식을 꼭 연습해볼 기회가 있으면 좋겠다.<br><br>
- ALS model을 학습시킬 때, 처음 실습 때와 동일하게 파라미터 중에서 factors를 100으로 놓고, iterations도 15로 놓고 진행했는데 생각보다 학습이 잘 안된 것 같은 느낌이 들어서, 각각 200과 20으로 늘려서 했더니 나쁘지 않은 학습이 진행된 것 같았다. factors는 두 특성 사이의 차원값이므로 더 높아질수록 더 정교한 결과가 나올 것이고, iterations도 학습 횟수이므로 더 좋은 학습 결과가 나올 것이다. 과적합이 되지 않는 선에서 적절한 값을 넣는 것이 필요한 것 같다.

#### 결과에 관하여
- 생각보다 결과로 나온 것들이 재미가 없었다. 내가 좋아하는 영화와 비슷한 영화들에 대해서도 그렇게 흥미로운 결과가 나온 것 같지 않고, 내가 좋아할 만한 영화를 추천 받는 것도 그냥 그저 그랬다. 확실히 영화들이 다 옛날 영화고 고전 영화들이 많아서 그런 것 같기도 하고, 또 다른 이유를 추측해보자면 처음에 나의 선호 영화 리스트를 정할 때 너무 뻔한 결과들이 나올만한 영화들을 선정해서 그런 것 같기도 하다. 또 영화 다섯 개를 지정할 때 다 비슷한 영화들로 정했어야 했는데, 내가 그나마 좋아하는 영화들로 채워넣으니깐 다섯 개의 영화가 일관성이 좀 부족했던 것 같다. 그래서 결과도 좀 애매하고 아리송하게 나온 것 같다. 물론 벡터 내적값도 높고 유사도도 어느 정도 괜찮은 정도로 나와서 프로젝트가 성공이라고 할 수 있겠지만, 전체적으로 만족스럽지는 않다.
<br><br>
- 더 최신 영화들과 더 많은 영화들로 이루어진 데이터셋으로, 선호 영화들 지정도 좀더 일관성 있게, 더 고도화된 모델로 한번 추천시스템을 제대로 실습할 수 있었으면 좋겠다.